In [1]:
from Environment.custom_env import CustomEnvironment
import json
import torch
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook

pygame 2.5.2 (SDL 2.28.3, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\srini\AppData\Local\Temp\ipykernel_4568\127627936.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [2]:
torch.cuda.is_available()

True

In [3]:
f = open('Environment/env_config.json')
env_variables = json.load(f)
f.close

<function TextIOWrapper.close()>

In [4]:
env = CustomEnvironment(env_config=env_variables)
# env = parallel_env(render_mode="human")
# env.reset(seed=42)

In [5]:
env.agents

[0, 1, 2]

In [6]:
episodes = 100

for episode in tqdm_notebook(range(episodes)): #episodes loop"

    curr_obs,infos = env.reset() #reset environment after each episode\n",

    losses = {}
    while(env.agents): #until there are any surviving agents 

        actions = {}
        log_probs = {}

        for i in range(env.n_agents): #objects contains all of the models, get the corresponding actions from each policy\n",
            
            if i in env.agents:
                act, log_prob = env.agents_objects[i].brain1.select_action(state = torch.tensor(curr_obs[i], dtype = torch.float32))
                actions[i] = act
                log_probs[i] = log_prob
            else:
                actions[i] = env.num_actions
                log_probs[i] = 69
            
        # print(actions)

        next_obs, rewards, terminations, truncations, infos = env.step(actions)
        
        # print(env.agents_objects)
        # print(curr_obs)
        # print(rewards)
        # print(terminations)
        # print(truncations)
        # print(infos)
        
        for i in range(env.n_agents):

            if i in env.agents:
                env.agents_objects[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                env.agents_objects[i].brain1.update_weights() 
            
            if i in env.justdie:
                env.justdie[i].brain1.push_to_buffer(torch.tensor(curr_obs[i], dtype = torch.float32),actions[i],rewards[i],torch.tensor(next_obs[i], dtype = torch.float32),log_probs[i],terminations[i])
                loss = env.justdie[i].brain1.update_weights() 
                del env.justdie[i]
                losses[i] = loss
        
        curr_obs = next_obs

    print(f"Episode {episode} end")
    print(f"Losses {losses}")
    print()

  0%|          | 0/100 [00:00<?, ?it/s]

agent0 consumed pellet129
agent1 consumed pellet166
agent2 consumed pellet142
agent0 consumed pellet123
agent2 consumed pellet124
agent1 consumed pellet93
agent1 consumed pellet206
agent2 consumed pellet7
agent0 died at time 191
agent2 consumed pellet177
agent1 died at time 212
agent2 died at time 233
Episode 0 end
Losses {0: tensor(16.2876), 1: tensor([[35.7174]]), 2: tensor(513.3079)}

agent2 consumed pellet255
agent0 consumed pellet296
agent1 consumed pellet103
agent2 consumed pellet50
agent0 consumed pellet124
agent2 consumed pellet51
agent1 died at time 170
agent2 consumed pellet25
agent0 died at time 191
agent2 died at time 233
Episode 1 end
Losses {1: tensor([[29.2808]]), 0: tensor(71.3849), 2: tensor(712.0789)}

agent0 consumed pellet33
agent0 consumed pellet131
agent2 consumed pellet27
agent1 consumed pellet279
agent0 consumed pellet144
agent0 consumed pellet41
agent1 consumed pellet38
agent2 died at time 170
agent1 died at time 191
agent0 died at time 233
Episode 2 end
Losses